In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
from traitlets import HasTraits, Unicode, observe, Instance, traitlets
import ipywidgets as widgets
import altair as alt
import os
import sys

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
savedir=os.getcwd()+'\\datatables\\'
script_dir = os.path.abspath(os.path.dirname(sys.argv[0]) or '.') #get the relative path

In [2]:
df=pd.read_parquet(f'{savedir}df_all_player.gzip')


In [5]:
class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, df, df_col, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        #Assign the dataframe to the class. This allows it to be used for filtering.
        self.df=df
        self.df_measures=df_col
        self.year_filter=None
        self.pl_filter=list(df.PLAYER_NAME.unique())
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))
        
    def get_year_filter(self,filter):
        self.year_filter=filter
        #revise the player filter list based on the seasons selected
        self.pl_filter=list(df[df.SEASON.isin(filter)].PLAYER_NAME.unique())
        

In [6]:
'''This section holds some of the fixed parameters used to set up the charts'''
df_col=['GAME_ID','TEAM_ABBREVIATION','PLAYER_NAME','FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS', 'GAME_DATE_EST', 'SEASON',
       'HM_AW', 'MIN_PLAY', 'SEC_PLAY', 'OPP', 'WINNER', 'PLAYER_WIN_OR_LOSE',
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'PACE','POSS', 'PIE']


value_dict={'Field Goals Made':'FGM',
'Field Goals Attempted':'FGA',
'Field Goal Percentage':'FG_PCT',
'Three Pointers Made':'FG3M',
'Three Pointers Attempted':'FG3A',
'Three Point Percentage':'FG3_PCT',
'Free Throws Made':'FTM',
'Free Throws Attempted':'FTA',
'Free Throw Percentage':'FT_PCT',
'Offensive Rebounds':'OREB',
'Defensive Rebounds':'DREB',
'Rebounds':'REB',
'Assists':'AST',
'Steals':'STL',
'Blocks':'BLK',
'Turnovers':'TO',
'Fouls':'PF',
'Points':'PTS',
'+/-':'PLUS_MINUS',
'Minutes Played':'MIN_PLAY',
'Seconds Played':'SEC_PLAY',
'Offensive Rating':'OFF_RATING',
'Defensive Rating':'DEF_RATING',
'Net Rating':'NET_RATING',
'Assist Percentage':'AST_PCT',
'Assist to Turnover Ratio':'AST_TOV',
'Assist Ratio':'AST_RATIO',
'Offensive Rebound Percentage':'OREB_PCT',
'Defensive Rebound Percentage':'DREB_PCT',
'Rebound Percentage':'REB_PCT',
'Team Turnover Percentage':'TM_TOV_PCT',
'Effective ':'EFG_PCT',
'True Shooting Percentage':'TS_PCT',
'Usage Percentage':'USG_PCT',
'Pace':'PACE',
'Possessions':'POSS',
'Player Impact Estimate':'PIE'}

'''A list of all the measures in the dataframe'''
value_list=['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS', 'MIN_PLAY', 'SEC_PLAY', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV',
       'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT',
       'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40',
       'POSS', 'PIE']


In [7]:
'''This is the code that creates the chart based on the players that are selected'''
def chart_selected(b):
    with out:
        out.clear_output()
        #load the dataframe
        source=w_year_button.df
        
        #store the parameters for the player(s) that are selected
        pl1=sel_pl1.value
        pl2=sel_pl2.value
        pl3=sel_pl3.value
        
        #select the x and y variables for the chart
        var1=value_dict[x_var.value]
        var2=value_dict[y_var.value]
        while True:
            if sel_pl1.value == 'Choose Someone' and sel_pl2.value == 'Choose Someone' and sel_pl3.value == 'Choose Someone':
                print('you need to select at least one player for the chart')
            elif var1==var2:
                print('please ensure the variables are different')
                break
            else:
                break
        
        source=source[(source.PLAYER_NAME==pl1) | (source.PLAYER_NAME==pl2) | (source.PLAYER_NAME==pl3)]
        chart= alt.Chart(source).mark_circle(size=60).encode(
        x=var1,
        y=var2,
        color='PLAYER_NAME',
        tooltip=['PLAYER_NAME', var1, var2,'SEASON']
        ).properties(
            width=1000,
            height=500
        ).interactive()
        display(chart);

In [8]:
'''This sets up the widget filters'''

'''This sets up the widgets for the year filters'''
button_layout=widgets.Layout(width='20%', height='80px',button_color='#FFFFFF')

out = widgets.Output() #this is required to capture values from the widgets themselves

#get the list of seasons in the dataset
year_list=list(df.SEASON.unique())

#the dropdown for the first season selector
w_fy=widgets.Dropdown(
value=year_list[0],
options=year_list,
description='First season of data')

#the dropdown for the last year widget
w_ly=widgets.Dropdown(
value=year_list[-1],
options=year_list,
description='Last season of data')

#add a button to lock in the year selections for the Dataframe
w_year_button=LoadedButton(df=df, df_col=value_list,
    description='Filter data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click this button to filter for players in the selected seasons',
    icon='check',
    layout=button_layout)



###If the first year in the dataset moves up then need to adjust the earliest year available for the last year and vice-versa
def fy_change(change): 
    with out:
        check=year_list.index(w_fy.value) 
        check2=year_list.index(w_ly.value)
        if check > check2: #if the value for the first year > then the last year selected make them equal to avoid an index error
            w_fy.value=w_ly.value
    
def ly_change(change):
    with out:
        check=year_list.index(w_fy.value)
        check2=year_list.index(w_ly.value)
        if check >= check2:
            w_ly.value=w_fy.value
        
    
    
    
#this runs the loop to update the options accordingly
w_fy.observe(fy_change, 'value')
w_ly.observe(ly_change, 'value')

text_year_tab= widgets.HTML('<h1 style="color:Tomato;">Use the filters below to determine what seasons of data to use:</h1>')
section_year_dropdown = widgets.VBox([w_fy,w_ly]) #set up the vertical box/parent for the year selectors
section_year= widgets.HBox([section_year_dropdown,w_year_button]) #use a horizontal container to add the button to the right of the year dropdowns

headings_year_tab=widgets.VBox([text_year_tab,section_year])

def on_button_clicked(b):
    with out:
        first_year=w_fy.value
        last_year=w_ly.value
        #Pull a list that stores the selected seasons to filter the dataframe
        year_filter=year_list[year_list.index(w_fy.value):year_list.index(w_ly.value)+1]
        w_year_button.get_year_filter(year_filter)
        #reset the player selections if the user changes the filter
        sel_pl1.options=w_year_button.pl_filter
        sel_pl2.options=sel_pl1.options
        sel_pl3.options=sel_pl1.options
        sel_pl1.value='Please select a player'
        sel_pl2.value='Please select a player'
        sel_pl3.value='Please select a player'
        print(f'Data covers games between {first_year} and {last_year} season(s) for {len(w_year_button.pl_filter)} players')
        tab.selected_index=1

#Call the on_button_clicked function when the user hits the filter button
w_year_button.on_click(on_button_clicked)

#pull the list of available players defaulting to all
PlayerList=list(df.PLAYER_NAME.unique())

#dropdown to select the first player
sel_pl1=widgets.Combobox(
value='Please select a player',
placeholder='Choose Someone',
options=PlayerList,
description='Choose Player 1:',
ensure_option=True,
disabled=False
)

#dropdown to select the second player
sel_pl2=widgets.Combobox(
value='Please select a player',
placeholder='Choose Someone',
options=PlayerList,
description='Choose Player 2:',
ensure_option=True,
disabled=False
)

#dropdown to select the third player
sel_pl3=widgets.Combobox(
value='Please select a player',
placeholder='Choose Someone',
options=PlayerList,
description='Choose Player 3:',
ensure_option=True,
disabled=False
)

measures=list(value_dict.keys())

#dropdown to select the x variable
x_var=widgets.Dropdown(
value=measures[0],
options=measures,
description='x-axis variable:',
ensure_option=True,
disabled=False
)

#dropdown to select the x variable
y_var=widgets.Dropdown(
value=measures[1],
options=measures,
description='y-axis variable:',
ensure_option=True,
disabled=False
)

#repopulate the charts if the x or y variables change
sel_pl1.observe(chart_selected)
sel_pl2.observe(chart_selected)
sel_pl3.observe(chart_selected)

#repopulate the charts if the x or y variables change
x_var.observe(chart_selected)
y_var.observe(chart_selected)


text_player_tab= widgets.HTML('<h1 style="color:Tomato;">Choose the players to compare:</h1>')

sel_pl1.layout.width='15' #spacing for the first player selection combobox
sel_pl2.layout.width='15' #spacing for the second player selection combobox

section_player_dropdown=widgets.VBox([sel_pl1,sel_pl2,sel_pl3,x_var,y_var]) #group the player selection widgets vertically

#section_players = widgets.VBox([text_player_tab,sel_pl1,sel_pl2]) #create the vertical box/parent for the player selection parameters
headings_players_tab=widgets.VBox([text_player_tab,section_player_dropdown])


list_widgets  = [headings_year_tab,headings_players_tab] #The widgets for tabs 1 and tab 2

tab=widgets.Tab()
tab.set_title(0,'Year Select')
tab.set_title(1,'Player Select')
tab.children=list_widgets



In [9]:
display(tab,out)

Output()

In [ ]:
w_fy.close(),w_ly.close(),out.close(),tab.close()

In [ ]:
savedir=os.getcwd()+'\\datatables\\'
df.to_parquet('{savedir}df_all_player.gzip',compression='gzip')

In [15]:
df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PLAYER_NAME',
       'START_POSITION', 'COMMENT', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS', 'GAME_DATE_EST', 'SEASON',
       'HM_AW', 'MIN_PLAY', 'SEC_PLAY', 'OPP', 'WINNER', 'PLAYER_WIN_OR_LOSE',
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE'],
      dtype='object')

In [21]:
df[['PLAYER_NAME','SEASON','POSS']][df['POSS']>2]

,PLAYER_NAME,SEASON,POSS
0,Scottie Pippen,1996-97,83.00
1,Dennis Rodman,1996-97,77.00
2,Luc Longley,1996-97,22.00
3,Ron Harper,1996-97,48.00
4,Michael Jordan,1996-97,86.00
...,...,...,...
10515,Landry Shamet,2019-20,69.00
10516,Lou Williams,2019-20,39.00
10517,Montrezl Harrell,2019-20,75.00
10518,Rodney McGruder,2019-20,26.00


0        False
1        False
2        False
3        False
4        False
         ...  
10518     True
10519     True
10520    False
10521    False
10522    False
Name: POSS, Length: 1010767, dtype: bool

In [3]:
df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GAME_DATE_EST,SEASON,HM_AW,MIN_PLAY,SEC_PLAY,OPP,WINNER,PLAYER_WIN_OR_LOSE,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0028000001,1610612756,PHX,76011,Alvan Adams,,,10.00,15.00,0.67,0.00,0.00,0.00,5.00,6.00,0.83,1.00,7.00,8.00,6.00,3.00,0.00,3.00,2.00,25.00,nan,1980-01-10,1980-1981,H,29.00,nan,GSW,H,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,0028000001,1610612756,PHX,78095,Alvin Scott,,,0.00,0.00,0.00,0.00,0.00,0.00,2.00,4.00,0.50,0.00,3.00,3.00,2.00,0.00,1.00,0.00,1.00,2.00,nan,1980-01-10,1980-1981,H,12.00,nan,GSW,H,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,0028000001,1610612756,PHX,77141,Dennis Johnson,,,10.00,17.00,0.59,0.00,0.00,0.00,8.00,9.00,0.89,4.00,1.00,5.00,3.00,2.00,0.00,2.00,3.00,28.00,nan,1980-01-10,1980-1981,H,31.00,nan,GSW,H,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,0028000001,1610612756,PHX,76436,Jeff Cook,,,2.00,3.00,0.67,0.00,0.00,0.00,0.00,0.00,0.00,1.00,6.00,7.00,1.00,1.00,0.00,3.00,3.00,4.00,nan,1980-01-10,1980-1981,H,33.00,nan,GSW,H,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,0028000001,1610612756,PHX,77308,Joel Kramer,,,2.00,2.00,1.00,0.00,0.00,0.00,2.00,4.00,0.50,1.00,1.00,2.00,2.00,0.00,0.00,0.00,1.00,6.00,nan,1980-01-10,1980-1981,H,16.00,nan,GSW,H,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [29]:
temp=df.groupby(['SEASON','GAME_ID'])['GAME_ID'].nunique()

In [33]:
temp.groupby('SEASON').sum()

SEASON
1980-1981     943
1981-1982     943
1982-1983     943
1983-1984     943
1984-1985     943
1985-1986     942
1986-1987     943
1987-1988     943
1988-1989    1025
1989-1990    1107
1990-91      1107
1991-92      1107
1992-93      1107
1993-94      1107
1994-95      1107
1995-96      1189
1996-97      1189
1997-98      1189
1998-99       725
1999-00      1189
2000-01      1189
2001-02      1189
2002-03      1189
2003-04      1189
2004-05      1230
2005-06      1230
2006-07      1230
2007-08      1230
2008-09      1230
2009-10      1230
2010-11      1230
2011-12       990
2012-13      1213
2013-14      1230
2014-15      1230
2015-16      1230
2016-17      1230
2017-18      1230
2018-19      1230
2019-20       419
Name: GAME_ID, dtype: int64

In [39]:
def total_games(season):
    if season == 1998:
        total_g=725
    elif season ==2011:
        total_g=1025
    elif season <=1987:
        total_g=943
    elif season <=1994:
        total_g=1107
    elif season <=1994:
        total_g=1189
    else:
        total_g=1230
    return total_g

In [42]:
for i in range(1980,2020):
    print(f'{i}:{total_games(i)}')

1980:943
1981:943
1982:943
1983:943
1984:943
1985:943
1986:943
1987:943
1988:1107
1989:1107
1990:1107
1991:1107
1992:1107
1993:1107
1994:1107
1995:1230
1996:1230
1997:1230
1998:725
1999:1230
2000:1230
2001:1230
2002:1230
2003:1230
2004:1230
2005:1230
2006:1230
2007:1230
2008:1230
2009:1230
2010:1230
2011:1025
2012:1230
2013:1230
2014:1230
2015:1230
2016:1230
2017:1230
2018:1230
2019:1230
